<a href="https://colab.research.google.com/github/sagarstormborn/star_gan_final_year/blob/main/1d_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install torchaudio
!pip install torchlibrosa

     |████████████████████████████████| 1.9MB 8.0MB/s 


In [3]:
import glob
import tqdm
import torchaudio





data_path=glob.glob("/content/drive/MyDrive/dataset/downsampled_data/*.wav")
data_s=[]
time_limit=2
rate=22050
for path in tqdm.tqdm(data_path):
  waveform, sample_rate=torchaudio.load(path)
  if len(waveform[0])/sample_rate >=time_limit:   # checking the if given sample is more then required time limit 
      data_s.append(path)

data_path=glob.glob('/content/drive/MyDrive/vcc2018_training/*/*.wav')
data_t=[]
time_limit=2
rate=22050
for path in tqdm.tqdm(data_path):
  waveform, sample_rate=torchaudio.load(path)
  if len(waveform[0])/sample_rate >=time_limit:   # checking the if given sample is more then required time limit 
      data_t.append(path)


100%|██████████| 972/972 [06:34<00:00,  2.46it/s]


In [4]:
from torch.utils.data import Dataset, DataLoader

class audio_data_set(Dataset):
  def __init__(self,path_for_target,path_for_source,time_limit=2,sample_rate=22050):
    self.list_t=path_for_target
    self.list_s=path_for_source
    self.lenth_t=len(self.list_t)
    self.lenth_s=len(self.list_s)
    self.sample_rate=sample_rate
    self.time_limit=time_limit
  def __len__(self):
    return max(self.lenth_t,self.lenth_s)
  
  def __getitem__(self, idx):
    path_t=self.list_t[idx % self.lenth_t]
    path_s=self.list_s[idx % self.lenth_s]
    
    waveform_t, sample_rate_t=torchaudio.load(path_t)
    waveform_s, sample_rate_s=torchaudio.load(path_s)
    
    return waveform_t[0][:self.sample_rate*self.time_limit].view(1,-1),waveform_s[0][:self.sample_rate*self.time_limit].view(1,-1)

In [5]:
!pip install torchinfo
from torchinfo import summary

In [6]:
from torch.utils.data import DataLoader
data_set=audio_data_set(path_for_target=data_t,path_for_source=data_s)
dataloader=DataLoader(data_set, batch_size=5, shuffle=True,num_workers=2)

In [11]:
"""
generator implementaion 
"""

import torch 
import torch.nn as nn 


class Generator(nn.Module):
  def __init__(self,audio_vector=44100,channels=1,features_g=10,BatchSize=1):
    super(Generator,self).__init__()
    self.net=nn.Sequential(
    nn.Conv1d
    (in_channels=channels, out_channels=16, kernel_size=4, stride=2, padding=0, bias=False, padding_mode="reflect"),
    nn.InstanceNorm1d(num_features=16),
    nn.LeakyReLU(negative_slope=0.02),   

    nn.Conv1d
    (in_channels=16, out_channels=32, kernel_size=4, stride=2, padding=1, bias=False, padding_mode="reflect"),
    nn.InstanceNorm1d(num_features=32), 
    nn.LeakyReLU(negative_slope=0.02),

    nn.Conv1d
    (in_channels=32, out_channels=64, kernel_size=4, stride=2, padding=1, bias=False, padding_mode="reflect"),
    nn.InstanceNorm1d(num_features=64),
    nn.LeakyReLU(negative_slope=0.02), 

    nn.Conv1d
    (in_channels=64, out_channels=128, kernel_size=4, stride=2, padding=1, bias=False, padding_mode="reflect"),
    nn.InstanceNorm1d(num_features=128),
    nn.LeakyReLU(negative_slope=0.02),
    

    nn.Conv1d
    (in_channels=128, out_channels=128, kernel_size=3, stride=1, padding=1, bias=False, padding_mode="reflect"),
    nn.InstanceNorm1d(num_features=128), 
    nn.LeakyReLU(negative_slope=0.02),

    nn.Conv1d
    (in_channels=128, out_channels=128, kernel_size=3, stride=1, padding=1, bias=False, padding_mode="reflect"),
    nn.InstanceNorm1d(num_features=128), 
    nn.LeakyReLU(negative_slope=0.02),

    nn.Conv1d
    (in_channels=128, out_channels=128, kernel_size=3, stride=1, padding=1, bias=False, padding_mode="reflect"),
    nn.InstanceNorm1d(num_features=128), 
    nn.LeakyReLU(negative_slope=0.02),

    nn.Conv1d
    (in_channels=128, out_channels=128, kernel_size=3, stride=1, padding=1, bias=False, padding_mode="reflect"),
    nn.InstanceNorm1d(num_features=128), 
    nn.LeakyReLU(negative_slope=0.02),
    
    nn.ConvTranspose1d
    (in_channels=128, out_channels=64, kernel_size=4, stride=2, padding=1, bias=False, padding_mode="zeros"),
    nn.InstanceNorm1d(num_features=64),
    nn.LeakyReLU(negative_slope=0.02),

    nn.ConvTranspose1d
    (in_channels=64, out_channels=32, kernel_size=4, stride=2, padding=1, bias=False, padding_mode="zeros"),
    nn.InstanceNorm1d(num_features=32),
    nn.LeakyReLU(negative_slope=0.02),

    nn.ConvTranspose1d
    (in_channels=32, out_channels=16, kernel_size=4, stride=2, padding=1, bias=False, padding_mode="zeros"),
    nn.InstanceNorm1d(num_features=16),
    nn.LeakyReLU(negative_slope=0.02),

    nn.ConvTranspose1d
    (in_channels=16, out_channels=1, kernel_size=6, stride=2, padding=0, bias=False, padding_mode="zeros"),
    nn.Tanh(),

    )
        
  def forward(self, a):
    return self.net(a) 

In [13]:
def initialize_weights(model):
    # Initializes weights according to the DCGAN paper
    for m in model.modules():
        if isinstance(m, (nn.Conv1d, nn.ConvTranspose1d)):
            nn.init.normal_(m.weight.data, 0.0, 0.02)


class Discriminator(nn.Module):
    def __init__(self):
        super(Discriminator, self).__init__()
        self.disc = nn.Sequential(
              # input: N x44100
            # nn.Unflatten(1, (1,44100)),
            
            self._block(1,1,42,21,0),
            self._block(1,1,42,21,0),
            self._block(1,1,9,9,0),
            nn.Flatten(),
            nn.Linear(in_features=10, out_features=1, bias=True),
            nn.Sigmoid(),
        )

    def _block(self, in_channels, out_channels, kernel_size, stride, padding):
        return nn.Sequential(
            nn.Conv1d(in_channels, out_channels, kernel_size, stride, padding,
                       dilation=1, groups=1, bias=False, padding_mode='zeros'),
            nn.BatchNorm1d(out_channels),
            nn.LeakyReLU(0.2),
        )

    def forward(self, x):
        return self.disc(x)
             

In [56]:
import torch
import torch.nn as nn
import torch.optim as optim

# Hyperparameters etc.
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
LEARNING_RATE = 2e-4  # could also use two lrs, one for gen and one for disc
BATCH_SIZE = 64
NUM_EPOCHS = 10

gen = Generator().to(device)
disc = Discriminator().to(device)
initialize_weights(gen)
initialize_weights(disc)

opt_gen = optim.Adam(gen.parameters(), lr=LEARNING_RATE, betas=(0.5, 0.999))
opt_disc = optim.Adam(disc.parameters(), lr=LEARNING_RATE, betas=(0.5, 0.999))
criterion = nn.BCELoss()

step = 0

gen.train()
disc.train()

Discriminator(
  (disc): Sequential(
    (0): Sequential(
      (0): Conv1d(1, 1, kernel_size=(42,), stride=(21,), bias=False)
      (1): BatchNorm1d(1, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): LeakyReLU(negative_slope=0.2)
    )
    (1): Sequential(
      (0): Conv1d(1, 1, kernel_size=(42,), stride=(21,), bias=False)
      (1): BatchNorm1d(1, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): LeakyReLU(negative_slope=0.2)
    )
    (2): Sequential(
      (0): Conv1d(1, 1, kernel_size=(9,), stride=(9,), bias=False)
      (1): BatchNorm1d(1, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): LeakyReLU(negative_slope=0.2)
    )
    (3): Flatten(start_dim=1, end_dim=-1)
    (4): Linear(in_features=10, out_features=1, bias=True)
    (5): Sigmoid()
  )
)

In [57]:
dataloader=DataLoader(data_set, batch_size=BATCH_SIZE, shuffle=True,num_workers=2)

import tqdm

# for epoch in tqdm.tqdm(range(NUM_EPOCHS)):
for epoch in range(NUM_EPOCHS):
  for batch_idx, (fake,real) in enumerate(dataloader):
    # print( batch_idx,real.shape,fake.shape)
    print(epoch,batch_idx)
    real = real.to(device)
    fake = fake.to(device)
    fake = gen(fake)


    ### Train Discriminator: max log(D(x)) + log(1 - D(G(z)))
    disc_real = disc(real).reshape(-1)
    loss_disc_real = criterion(disc_real, torch.ones_like(disc_real))
    disc_fake = disc(fake.detach()).reshape(-1)
    loss_disc_fake = criterion(disc_fake, torch.zeros_like(disc_fake))
    loss_disc = (loss_disc_real + loss_disc_fake) / 2
    disc.zero_grad()
    loss_disc.backward()
    opt_disc.step()

    ### Train Generator: min log(1 - D(G(z))) <-> max log(D(G(z))
    output = disc(fake).reshape(-1)
    loss_gen = criterion(output, torch.ones_like(output))
    gen.zero_grad()
    loss_gen.backward()
    opt_gen.step()

0 0
0 1
0 2
0 3
0 4
0 5
0 6
0 7
0 8
0 9
0 10
0 11
0 12
0 13
0 14
0 15
0 16
0 17
0 18
0 19
0 20
0 21
0 22
0 23
0 24
0 25
0 26
0 27
0 28
0 29
0 30
0 31
0 32
0 33
0 34
0 35
0 36
1 0
1 1
1 2
1 3
1 4
1 5
1 6
1 7
1 8
1 9
1 10
1 11
1 12
1 13
1 14
1 15
1 16
1 17
1 18
1 19
1 20
1 21
1 22
1 23
1 24
1 25
1 26
1 27
1 28
1 29
1 30
1 31
1 32
1 33
1 34
1 35
1 36
2 0
2 1
2 2
2 3
2 4
2 5
2 6
2 7
2 8
2 9
2 10
2 11
2 12
2 13
2 14
2 15
2 16
2 17
2 18
2 19
2 20
2 21
2 22
2 23
2 24
2 25
2 26
2 27
2 28
2 29
2 30
2 31
2 32
2 33
2 34
2 35
2 36
3 0
3 1
3 2
3 3
3 4
3 5
3 6
3 7
3 8
3 9
3 10
3 11
3 12
3 13
3 14
3 15
3 16
3 17
3 18
3 19
3 20
3 21
3 22
3 23
3 24
3 25
3 26
3 27
3 28
3 29
3 30
3 31
3 32
3 33
3 34
3 35
3 36
4 0
4 1
4 2
4 3
4 4
4 5
4 6
4 7
4 8
4 9
4 10
4 11
4 12
4 13
4 14
4 15
4 16
4 17
4 18
4 19
4 20
4 21
4 22
4 23
4 24
4 25
4 26
4 27
4 28
4 29
4 30
4 31
4 32
4 33
4 34
4 35
4 36
5 0
5 1
5 2
5 3
5 4
5 5
5 6
5 7
5 8
5 9
5 10
5 11
5 12
5 13
5 14
5 15
5 16
5 17
5 18
5 19
5 20
5 21
5 22
5 23
5 24
5 25
5 26


In [50]:

from IPython.display import Audio, display
def play_audio(audio):
  display(Audio(audio, rate=22050))


In [51]:
a,b=data_set[8]
play_audio(a)
play_audio(b)

In [59]:
result=gen(a.view(1,1,-1).to(device)).view(1,-1).to("cpu")
play_audio(result.detach().numpy())

In [87]:
gen=Generator()

In [64]:
summary(Generator(), input_size=(5,1,44100))

Layer (type:depth-idx)                   Output Shape              Param #
├─Sequential: 1-1                        [5, 1, 44100]             --
|    └─Conv1d: 2-1                       [5, 16, 22049]            64
|    └─InstanceNorm1d: 2-2               [5, 16, 22049]            --
|    └─LeakyReLU: 2-3                    [5, 16, 22049]            --
|    └─Conv1d: 2-4                       [5, 32, 11024]            2,048
|    └─InstanceNorm1d: 2-5               [5, 32, 11024]            --
|    └─LeakyReLU: 2-6                    [5, 32, 11024]            --
|    └─Conv1d: 2-7                       [5, 64, 5512]             8,192
|    └─InstanceNorm1d: 2-8               [5, 64, 5512]             --
|    └─LeakyReLU: 2-9                    [5, 64, 5512]             --
|    └─Conv1d: 2-10                      [5, 128, 2756]            32,768
|    └─InstanceNorm1d: 2-11              [5, 128, 2756]            --
|    └─LeakyReLU: 2-12                   [5, 128, 2756]            --
|    